# PageRank

In this notebook, we will use both NetworkX and cuGraph to compute the PageRank of each vertex in our test dataset.  The NetworkX and cuGraph processes will be interleaved so that each step can be compared.

Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Last Edit: 04/30/2019

RAPIDS Versions: 0.7.0    

Test Hardware

* GP100 32G, CUDA 9.2


## Introduction
Pagerank is measure of the relative importance, also called centrality, of a vertex based on the relative importance of it's neighbors.  PageRank was developed by Google and is (was) used to rank it's search results. PageRank uses the connectivity information of a graph to rank the importance of each vertex. 

See [Wikipedia](https://en.wikipedia.org/wiki/PageRank) for more details on the algorithm.

To compute the Pagerank scores for a graph in cuGraph we use:<br>

**cugraph.pagerank(G,alpha=0.85, max_iter=100, tol=1.0e-5)**
* __G__: cugraph.Graph object
* __alpha__: float, The damping factor represents the probability to follow an outgoing edge. default is 0.85
* __max_iter__: int, The maximum number of iterations before an answer is returned. This can be used to limit the execution time and do an early exit before the solver reaches the convergence tolerance. If this value is lower or equal to 0 cuGraph will use the default value, which is 100
* __tol__: float, Set the tolerance the approximation, this parameter should be a small magnitude value. The lower the tolerance the better the approximation. If this value is 0.0f, cuGraph will use the default value which is 0.00001. Setting too small a tolerance can lead to non-convergence due to numerical roundoff. Usually values between 0.01 and 0.00001 are acceptable.

Returns:
* __df__: a cudf.DataFrame object with two columns:
    * df['vertex']: The vertex identifier for the vertex
    * df['pagerank']: The pagerank score for the vertex
    
<br><br>
## cuGraph 0.7 Notice 
cuGraph version 0.7 has some limitations:
* Only Int32 Vertex ID are supported
* Only float (FP32) edge data is supported
* Vertex numbering is assumed to start at zero

These limitations are being addressed and will be fixed future versions.  
These example notebooks will illustrate how to manipulate the data so that it comforms to the current limitations    

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](./img/zachary_black_lines.png)


### Prep

In [ ]:
# This automatically time every cell's execution
!pip install ipython-autotime
%load_ext autotime

In [ ]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

In [ ]:
# NetworkX libraries
import community
import networkx as nx
from scipy.io import mmread

### Some Prep

In [ ]:
# define the parameters 
max_iter = 100  # The maximum number of iterations
tol = 0.00001   # tolerance
alpha = 0.85    # alpha

In [ ]:
# Define the path to the test data  
datafile='./data/karate-data.csv'

---
# NetworkX

In [ ]:
# Read the data, this also created a Graph 
file = open(datafile, 'rb')
Gnx = nx.read_edgelist(file)

In [ ]:
pr_nx = nx.pagerank(Gnx, alpha=alpha, max_iter=max_iter, tol=tol)

In [ ]:
pr_nx

# cuGraph

### Read in the data - GPU
The data file contains an edge list, which represents the connection of a vertex to another.  The `source` to `destination` pairs is in what is known as Coordinate Format (COO).  In this test case, the data is just two columns.  However a third, `weight`, column is also possible

In [ ]:
# Test file    
datafile='./data/karate-data.csv'

# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()) )

### Create a Graph 

In [ ]:
# create a Graph using the source (src) and destination (dst) vertex pairs from the Dataframe 
G = cugraph.Graph()
G.add_edge_list(gdf["src"], gdf["dst"])

### Call the PageRank algorithm

In [ ]:
# Call cugraph.pagerank to get the pagerank scores
gdf_page = cugraph.pagerank(G)

_It was that easy!_  Let's now look at the results

In [ ]:
# Find the most important vertex using the scores
# This methods should only be used for small graph
bestScore = gdf_page['pagerank'][0]
bestVert = gdf_page['vertex'][0]

for i in range(len(gdf_page)):
    if gdf_page['pagerank'][i] > bestScore:
        bestScore = gdf_page['pagerank'][i]
        bestVert = gdf_page['vertex'][i]
        
print("Best vertex is " + str(bestVert) + " with score of " + str(bestScore))

In [ ]:
# A better way to do that would be to find the max and then use that values in a query
pr_max = gdf_page['pagerank'].max()

In [ ]:
filtered = gdf_page.query('pagerank >= @pr_max')
    
for i in range(len(filtered)):
    print("Best vertex is " + str(filtered['vertex'][i]) + 
        " with score of " + str(filtered['pagerank'][i]))             

___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___